#### Finetuning BERT for `Extractive Question Answering` 

We will finetune a BERT model on the task of extractive QA, which involves taking a `factoid question` and a `context passage` of text and `labeling a span` of text from that passage which contains the `answer`. We can frame this as a `classification task`. First we concatenate the question and context passage pair, seperated by a `[SEP]` token. Then we compute the BERT encoding for this sequence. Then we apply a linear transform to each output token's encoding vector to compute a scalar score. By passing the scores from all tokens through a softmax, we obtain a `probability distribution` over tokens in the sequence, which we can interpret as the probability of a token being the start of the span. We actually will compute two separate linear transforms of all tokens and pass both sets of scores through a softmax to get two probability distributions over tokens, one for `start of span` and one for `end of span`. 

We will train this model on the SQuAD v1 dataset which contains passages with multiple questions and answer span pairs. We will use the cross entropy loss at the softmax output. To make predictions, we can simply just add up the scores of the `ith` token being the start and the `jth` token being the end for all i and j>i, then declare the (i,j) with the highest score as the predicted span.





In [1]:
import torch
from transformers import BertTokenizerFast, BertModel
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import csv
import random
from tqdm import tqdm
import psutil
import json
import wandb
wandb.login()

print(torch.cuda.is_available())

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tanzids. Use `wandb login --relogin` to force relogin


True


First, let's load the data from file and then set up pytorch datasets

In [2]:
# load the train and dev JSON documents
with open("train.json", "r") as train_file:
    squad_train = json.load(train_file)         
with open("dev.json", "r") as dev_file:
    squad_dev = json.load(dev_file) 

In [3]:
def get_passages(squad, num_titles=None):
    if num_titles is None:
        num_titles = len(squad['data'])
    # for each title, get passages and all corresponding questions from SQuAD train set
    passages = []
    questions = []
    num_questions = 0
    for i in range(num_titles):
        #print(f"Title# {i}: {squad['data'][i]['title']}, Number of passages: {len(squad['data'][i]['paragraphs'])}")
        for p in squad['data'][i]['paragraphs']:
            # we will append the title to each passage and question
            passages.append(p['context'])
            questions.append([q for q in p['qas']])    
            num_questions += len(p['qas'])
    print(f"Number of passages: {len(passages)}")
    print(f"Number of questions: {num_questions}")
    return passages, questions

In [4]:
passages_train, questions_train = get_passages(squad_train)
passages_val, questions_val = get_passages(squad_dev)

Number of passages: 19035
Number of questions: 130319
Number of passages: 1204
Number of questions: 11873


In [5]:
passage_lengths = [len(p) for p in passages_train]
print(f"Max passage length: {max(passage_lengths)}, Avg passage length: {sum(passage_lengths)/len(passage_lengths)}")

Max passage length: 3706, Avg passage length: 735.5478854741266


Note that the context passaged are very long (over 700 words on average) and won't fit into our BERT model (which can only take upto 512 tokens per sequence). So we will instead take a shorter fixed size context window for each question.  

In [6]:
q = questions_train[16][0]
answer_start_pos = q['answers'][0]['answer_start']
answer_end_pos = answer_start_pos + len(q['answers'][0]['text'])
context = passages_train[16]

print(f"Question: {q['question']}")
print(f"Answer span: {context[answer_start_pos:answer_end_pos]}")

Question: How many awards was Beyonce nominated for at the 52nd Grammy Awards?
Answer span: ten


In [7]:
window_size_chars = 500
# pick a random context window around the answer (try to keep at least 40% of the characters in window on the left side of the answer)
answer_middle_pos = int((answer_start_pos+answer_end_pos)/2) 
a = max(0,answer_end_pos-window_size_chars)
b = max(0, answer_start_pos - 0.4*window_size_chars)
random_window_start_pos = random.randint(a, b)
#window_start_pos = max(0,answer_middle_pos-window_size_chars)
#window_end_pos = answer_middle_pos+window_size_chars
window_start_pos = random_window_start_pos
window_end_pos = window_start_pos+window_size_chars

context_window = context[window_start_pos:window_end_pos]
print("Context window: ", context_window)

answer_start_pos_window = answer_start_pos - window_start_pos
answer_end_pos_window = answer_start_pos_window + len(q['answers'][0]['text'])
answer_window = context_window[answer_start_pos_window:answer_end_pos_window]
print(f"Answer window: {answer_window}")

# Trim off stray partial words at the beginning and end
context_window_words = context_window.split()
# only trim if the first stray word does not overlap with the answer span
if answer_start_pos_window > len(context_window_words[0]):
    context_window = ' '.join(context_window_words[1:-1])
    left_trim_length = len(context_window_words[0]) + 1 # add 1 for the white space between stary partial first word and next word
    answer_start_pos_window = answer_start_pos_window - left_trim_length
    answer_end_pos_window = answer_end_pos_window - left_trim_length

answer_window = context_window[answer_start_pos_window:answer_end_pos_window]
print(f"Answer window trimmed: {answer_window}")

Context window:  At the 52nd Annual Grammy Awards, Beyoncé received ten nominations, including Album of the Year for I Am... Sasha Fierce, Record of the Year for "Halo", and Song of the Year for "Single Ladies (Put a Ring on It)", among others. She tied with Lauryn Hill for most Grammy nominations in a single year by a female artist. In 2010, Beyoncé was featured on Lady Gaga's single "Telephone" and its music video. The song topped the US Pop Songs chart, becoming the sixth number-one for both Beyoncé and Gaga,
Answer window: ten
Answer window trimmed: ten


Since we will use WordPiece tokenization, we need to be careful about converting the character positions of the start and end of the span to subwork token positions.

In [8]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [9]:
# encode the passage
context_encoded = tokenizer.encode_plus((context_window, q['question']), add_special_tokens=True, return_offsets_mapping=True)
print(context_encoded.keys())
# convert character positions from original sentence to subword token positions
start_pos_enc = context_encoded.char_to_token(answer_start_pos_window)
end_pos_enc = context_encoded.char_to_token(answer_end_pos_window-1)
# get the corresponding subword token span
answer_span_encoded = context_encoded['input_ids'][start_pos_enc:end_pos_enc+1]
# decode the span to check if it matches original answer span
print(f"Decoded subword token span: {tokenizer.decode(answer_span_encoded)}")
print(f"Decoded sentence pair: {tokenizer.decode(context_encoded['input_ids'])}")

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping'])
Decoded subword token span: ten
Decoded sentence pair: [CLS] the 52nd annual grammy awards, beyonce received ten nominations, including album of the year for i am... sasha fierce, record of the year for " halo ", and song of the year for " single ladies ( put a ring on it ) ", among others. she tied with lauryn hill for most grammy nominations in a single year by a female artist. in 2010, beyonce was featured on lady gaga's single " telephone " and its music video. the song topped the us pop songs chart, becoming the sixth number - one for both beyonce and [SEP] how many awards was beyonce nominated for at the 52nd grammy awards? [SEP]


Now let's create question, trimmed context window and answer span instances. Note: some quesitions may contain empty answers list, those will be skipped.

In [30]:
# function for creating instances
def get_instances(passages, questions, window_size_chars=400):
    triplets = []
    for i, context in enumerate(passages):
        for q in questions[i]:
            for a in q['answers']:
                # get start and end character positions of answer span 
                answer_start_pos = a['answer_start']
                answer_end_pos = answer_start_pos + len(a['text'])
                # get windowed context
                a = max(0,answer_end_pos-window_size_chars)
                b = max(0, answer_start_pos - 0.4*window_size_chars)
                window_start_pos = random.randint(a, b)
                window_end_pos = window_start_pos+window_size_chars
                context_window = context[window_start_pos:window_end_pos]
                # offset the answer span positions
                answer_start_pos_window = answer_start_pos - window_start_pos
                answer_end_pos_window = answer_start_pos_window + len(q['answers'][0]['text'])
                # Trim off stray partial words at the beginning and end of window
                context_window_words = context_window.split()
                # only trim if the first stray word does not overlap with the answer span
                """
                if answer_start_pos_window > len(context_window_words[0]):
                    context_window = ' '.join(context_window_words[1:-1])
                    left_trim_length = len(context_window_words[0]) + 1 # add 1 for the white space between stary partial first word and next word
                    answer_start_pos_window = answer_start_pos_window - left_trim_length
                    answer_end_pos_window = answer_end_pos_window - left_trim_length
                """    
                if (answer_start_pos_window < 0) or (answer_end_pos_window < 0) or (answer_end_pos_window < answer_start_pos_window):
                    print(f"Context: {context}")
                    print(f"Question: {q}")
                    print(f"answer_start_pos: {answer_start_pos}, answer_end_pos: {answer_end_pos}")    
                    print(f"context char length: {len(context)}, window start pos: {window_start_pos}, window end pos: {window_end_pos}")
                    print(f"answer start pos window: {answer_start_pos_window}, answer end pos window: {answer_end_pos_window}")    
                    print(f"Context trimmed: {context_window}")
                    raise Exception(f"Trimmed answer span is negative! start_pos: {answer_start_pos_window}, end_pos: {answer_end_pos_window}")
                
                triplets.append(((context_window, q['question']), answer_start_pos_window, answer_end_pos_window))
    return triplets            

In [31]:
triplets_train = get_instances(passages_train, questions_train)
triplets_val = get_instances(passages_val, questions_val)

In [32]:
triplets_train[37]

(("d performances have led to critics hailing her as one of the best entertainers in contemporary popular music. Throughout a career spanning 19 years, she has sold over 118 million records as a solo artist, and a further 60 million with Destiny's Child, making her one of the best-selling music artists of all time. She has won 20 Grammy Awards and is the most nominated woman in the award's history. T",
  'How many Grammy awards has Beyoncé won?'),
 326,
 328)

Now let's set up a pytorch dataset

In [33]:
class SquadDataset(Dataset):
    def __init__(self, data, max_length=256):
        self.data = data
        self.tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # get the data instance
        sentence_pair = self.data[idx][0]
        start_pos_char = self.data[idx][1]
        end_pos_char = self.data[idx][2]
        # encode the sentences
        encoded = self.tokenizer.encode_plus(
            sentence_pair,
            add_special_tokens=True,
            return_offsets_mapping=True,
            truncation=True,
            padding="max_length",  # Pad to the maximum length
            max_length=self.max_length  # Set the maximum length
        )
        # get the token indices and attention mask
        input_idx = encoded['input_ids']
        attn_mask = encoded['attention_mask']
        # convert start and end character positions to subword token positions
        start_pos_enc = encoded.char_to_token(start_pos_char)
        end_pos_enc = encoded.char_to_token(end_pos_char-1)
        if (start_pos_enc is None) or (end_pos_enc is None):
            print(f"start pos char: {start_pos_char}, end pos char: {end_pos_char}")
            print(f"decoded sentence pair: {self.tokenizer.decode(input_idx)}")
            print(f"answer span: {sentence_pair[0][start_pos_char:end_pos_char]}")
            raise Exception(f"Start or end position is None! start_pos: {start_pos_enc}, end_pos: {end_pos_enc}")
        if len(input_idx) != self.max_length:
            raise Exception(f"Encoded input sequence length {len(input_idx)} is not equal to max_length {self.max_length}!")

        # convert to tensors
        input_idx = torch.tensor(input_idx)
        attn_mask = torch.tensor(attn_mask)
        start_pos_enc = torch.tensor(start_pos_enc)
        end_pos_enc = torch.tensor(end_pos_enc)
        return input_idx, start_pos_enc, end_pos_enc, attn_mask

In [34]:
train_dataset = SquadDataset(triplets_train)
val_dataset = SquadDataset(triplets_val)

#### Now define the classification model.

In [61]:
class BERTExtractiveQA(torch.nn.Module):
    def __init__(self, hidden_size=768, dropout_rate=0.1, finetune=False):
        super().__init__()
        # load pretrained BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = torch.nn.Dropout(dropout_rate)      
        # define two classifier heads, one for predicting start of span and another for end of span 
        self.classifier_head_start_span = torch.nn.Linear(hidden_size, 1)
        self.classifier_head_end_span = torch.nn.Linear(hidden_size, 1)

        for param in self.bert.parameters():
            if finetune:
                # make all parameters of BERT model trainable if we're finetuning
                param.requires_grad = True
            else:
                # freeze all parameters of BERT model if we're not finetuning
                param.requires_grad = False

    def forward(self, input_idx, labels_start, labels_end, attn_mask):
        # compute BERT encodings
        bert_output = self.bert(input_idx, attention_mask=attn_mask)
        bert_output = bert_output.last_hidden_state # shape: (batch_size, sequence_length, hidden_size)
        # compute logits/scores over tokens for each of the classifier heads
        logits_start = self.classifier_head_start_span(bert_output).squeeze(-1)  # shape: (batch_size, sequence_length)
        logits_end = self.classifier_head_end_span(bert_output).squeeze(-1)  # shape: (batch_size, sequence_length)
        # compute loss
        loss = F.cross_entropy(logits_start, labels_start) + F.cross_entropy(logits_end, labels_end) 

        return logits_start, logits_end, loss
    

# training loop
def train(model, optimizer, train_dataloader, val_dataloader, scheduler=None, device="cpu", num_epochs=10, val_every=1, save_every=None, log_metrics=None):
    avg_loss = 0
    train_acc = 0
    val_loss = 0
    val_acc = 0
    model.train()
    for epoch in range(num_epochs):
        num_correct = 0
        num_total = 0
        pbar = tqdm(train_dataloader, desc="Epochs")
        for batch in pbar:
            inputs, targets_start, targets_end, attn_mask = batch
            # move batch to device
            inputs, targets_start, targets_end, attn_mask = inputs.to(device), targets_start.to(device), targets_end.to(device), attn_mask.to(device)
            # forward pass
            logits_start, logits_end, loss = model(inputs, targets_start, targets_end, attn_mask)
            # reset gradients
            optimizer.zero_grad()
            # backward pass
            loss.backward()
            # optimizer step
            optimizer.step()
            avg_loss = 0.9* avg_loss + 0.1*loss.item()
            B, _ = inputs.shape
            y_pred_start = logits_start.argmax(dim=-1).view(-1) # shape (B,)
            y_pred_end = logits_end.argmax(dim=-1).view(-1) # shape (B,)
            num_correct += ((y_pred_start.eq(targets_start.view(-1)) + y_pred_end.eq(targets_end.view(-1))) == 2).sum().item()            
            num_total += B
            train_acc = num_correct / num_total        
            
            pbar.set_description(f"Epoch {epoch + 1}, EMA Train Loss: {avg_loss:.3f}, Train Accuracy: {train_acc: .3f}, Val Loss: {val_loss: .3f}, Val Accuracy: {val_acc: .3f}")  

            if log_metrics:
                metrics = {"Batch loss" : loss.item(), "Moving Avg Loss" : avg_loss, "Val Loss": val_loss}
                log_metrics(metrics)

        if scheduler is not None:
            scheduler.step()
        
        if val_every is not None:
            if epoch%val_every == 0:
                # compute validation loss
                val_loss, val_acc = validation(model, val_dataloader, device=device)
                pbar.set_description(f"Epoch {epoch + 1}, EMA Train Loss: {avg_loss:.3f}, Train Accuracy: {train_acc: .3f}, Val Loss: {val_loss: .3f}, Val Accuracy: {val_acc: .3f}") 

        if save_every is not None:
            if (epoch+1) % save_every == 0:
                save_model_checkpoint(model, optimizer, epoch, avg_loss)


def validation(model, val_dataloader, device="cpu"):
    model.eval()
    val_losses = torch.zeros(len(val_dataloader))
    with torch.no_grad():
        num_correct = 0
        num_total = 0
        for i,batch in enumerate(val_dataloader):
            inputs, targets_start, targets_end, attn_mask = batch
            inputs, targets_start, targets_end, attn_mask = inputs.to(device), targets_start.to(device), targets_end.to(device), attn_mask.to(device)
            logits_start, logits_end, loss = model(inputs, targets_start, targets_end, attn_mask)
            B, _ = inputs.shape
            y_pred_start = logits_start.argmax(dim=-1).view(-1) # shape (B,)
            y_pred_end = logits_end.argmax(dim=-1).view(-1) # shape (B,)
            num_correct += ((y_pred_start.eq(targets_start.view(-1)) + y_pred_end.eq(targets_end.view(-1))) == 2).sum().item()            
            num_total += B
            val_losses[i] = loss.item()
    model.train()
    val_loss = val_losses.mean().item()
    val_accuracy = num_correct / num_total
    return val_loss, val_accuracy


def save_model_checkpoint(model, optimizer, epoch=None, loss=None, filename=None):
    # Save the model and optimizer state_dict
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }

    # Save the checkpoint to a file
    if filename:
        torch.save(checkpoint, filename)
    else:
        torch.save(checkpoint, 'qa_checkpoint.pth')
    print(f"Saved model checkpoint!")


def load_model_checkpoint(model, optimizer, filename=None):
    if filename:
        checkpoint = torch.load(filename)
    else:
        checkpoint = torch.load('qa_checkpoint.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    model.train()
    print("Loaded model from checkpoint!")
    return model, optimizer             

Now let's train the model with and without finetuning

In [62]:
block_size = 256
B = 64
DEVICE = "cuda"
learning_rate = 5e-3


train_dataloader = DataLoader(train_dataset, batch_size=B, shuffle=True) #, pin_memory=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=B, shuffle=True) #, pin_memory=True, num_workers=2)

# model with finetuning disabled
model = BERTExtractiveQA(finetune=False).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler =  torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.95)
#model, optimizer = load_model_checkpoint(model, optimizer)

num_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters in transformer network: {num_params/1e6} M")
print(f"RAM used: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")


Total number of parameters in transformer network: 109.483778 M
RAM used: 6661.90 MB


In [41]:
inputs, targets_start, targets_end, attn_mask = next(iter(train_dataloader))

print(inputs.shape, targets_start.shape, targets_end.shape, attn_mask.shape)

torch.Size([64, 256]) torch.Size([64]) torch.Size([64]) torch.Size([64, 256])


In [60]:
train(model, optimizer, train_dataloader, val_dataloader, device=DEVICE, num_epochs=1, save_every=50, val_every=1) 

Epochs:   0%|          | 0/1357 [00:00<?, ?it/s]

logits start shape: torch.Size([64, 256]), labels start shape: torch.Size([64])
logits end shape: torch.Size([64, 256]), labels end shape: torch.Size([64])


Epoch 1, EMA Train Loss: 1.113, Train Accuracy:  0.000, Val Loss:  0.000, Val Accuracy:  0.000:   0%|          | 1/1357 [00:00<11:56,  1.89it/s]

logits start shape: torch.Size([64, 256]), labels start shape: torch.Size([64])
logits end shape: torch.Size([64, 256]), labels end shape: torch.Size([64])


Epoch 1, EMA Train Loss: 2.055, Train Accuracy:  0.000, Val Loss:  0.000, Val Accuracy:  0.000:   0%|          | 2/1357 [00:01<11:34,  1.95it/s]

...

logits start shape: torch.Size([64, 256]), labels start shape: torch.Size([64])
logits end shape: torch.Size([64, 256]), labels end shape: torch.Size([64])


Epoch 1, EMA Train Loss: 4.091, Train Accuracy:  0.000, Val Loss:  0.000, Val Accuracy:  0.000:   0%|          | 5/1357 [00:02<11:24,  1.98it/s]

logits start shape: torch.Size([64, 256]), labels start shape: torch.Size([64])
logits end shape: torch.Size([64, 256]), labels end shape: torch.Size([64])


Epoch 1, EMA Train Loss: 4.091, Train Accuracy:  0.000, Val Loss:  0.000, Val Accuracy:  0.000:   0%|          | 5/1357 [00:03<13:45,  1.64it/s]


KeyboardInterrupt: 